In [1]:
import json
import re
import random
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

rows = []
def filter_tweets(tweets, classification):
    for tweet in tweets:
        currenttext = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet['text']).split())
        #currenttext = tweet['text']
        rows.append({'text': currenttext, 'class': classification})
    

with open('allbottweets.json') as data_file:
    filter_tweets(json.load(data_file), 1)
    
with open('allnonbottweets.json') as data_file:
    filter_tweets(json.load(data_file), 0)

random.shuffle(rows)
data = DataFrame(rows)

In [2]:
#BERNOULLI

import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
targets = data['class']

Data_train, Data_test, target_train, target_test = train_test_split(data['text'], targets, test_size=0.2, random_state=0)

from sklearn.pipeline import Pipeline
text_clf_mb = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', BernoulliNB()),])
clf_mb = text_clf_mb.fit(Data_train, target_train)

predicted_mb = clf_mb.predict(Data_test)
np.mean(predicted_mb == target_test)

0.73999999999999999

In [3]:
#Precision, recall and F1
from sklearn.metrics import classification_report
target_names = ['bot', 'nonbot']
print(classification_report(target_test, predicted_mb, target_names=target_names))

#AUC 
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(target_test, predicted_mb, pos_label=1)
print(fpr)
print(tpr)
print(metrics.auc(fpr, tpr))

             precision    recall  f1-score   support

        bot       0.85      0.62      0.72        53
     nonbot       0.67      0.87      0.76        47

avg / total       0.76      0.74      0.74       100

[ 0.          0.37735849  1.        ]
[ 0.          0.87234043  1.        ]
0.747490967483


In [4]:
#MULTINOMIAL

import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
targets = data['class']

Data_train, Data_test, target_train, target_test = train_test_split(data['text'], targets, test_size=0.2, random_state=0)

from sklearn.pipeline import Pipeline
text_clf_mb = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])
clf_mb = text_clf_mb.fit(Data_train, target_train)

predicted_mb = clf_mb.predict(Data_test)
np.mean(predicted_mb == target_test)

0.76000000000000001

In [5]:
#Precision, recall and F1
from sklearn.metrics import classification_report
target_names = ['bot', 'nonbot']
print(classification_report(target_test, predicted_mb, target_names=target_names))

#AUC 
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(target_test, predicted_mb, pos_label=1)
print(fpr)
print(tpr)
print(metrics.auc(fpr, tpr))

             precision    recall  f1-score   support

        bot       0.75      0.83      0.79        53
     nonbot       0.78      0.68      0.73        47

avg / total       0.76      0.76      0.76       100

[ 0.          0.16981132  1.        ]
[ 0.          0.68085106  1.        ]
0.755519871538
